In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'
import sys

import pandas as pd
import numpy as np

import time

import torch

import copy

from tqdm.auto import tqdm

In [2]:
from datasets import load_original_dataset, load_deleted_dataset
from models import CNN

In [3]:
DATA_DIR = 'Datasets/Features/'
BATCH_SIZE = 32
EPOCHS = 5
PERCENTAGES = [1, 10, 20, 30, 40, 50, 60, 70, 80, 90, 99]

In [4]:
os.makedirs('results/kpriors', exist_ok=True)

In [5]:
use_cuda = True

# https://github.com/team-approx-bayes/kpriors/blob/main/main.py#L71
learning_rate = 0.005

# https://github.com/team-approx-bayes/kpriors/blob/main/main.py#L20
adaptation_task = 'remove_data'

# https://github.com/team-approx-bayes/kpriors/blob/main/main.py#L83
prior_prec = 5

# https://github.com/team-approx-bayes/kpriors/blob/main/main.py#L31
adaptation_methods = ['Replay','K-priors']

In [6]:
sys.path.append(os.path.abspath('./libraries/kpriors/'))

# https://github.com/team-approx-bayes/kpriors/blob/main/main.py#L114
from adamreg import AdamReg
import utils
import train

In [7]:
# https://github.com/team-approx-bayes/kpriors/blob/main/models.py#L75

# Return all parameters as a vector
def return_parameters(model):
    num_params = sum([np.prod(p.size()) for p in model.parameters()])
    means = torch.zeros(num_params)

    start_ind = 0
    for p in model.parameters():
        num = np.prod(p.size())
        means[start_ind:start_ind+num] = p.data.reshape(-1)
        start_ind += num

    return means

In [8]:
def fit(base_model, save_dir, train_set, test_set, forget_set):
    
    os.makedirs(save_dir, exist_ok=True)
    
    # https://github.com/team-approx-bayes/kpriors/blob/main/train.py#L10
    error = torch.nn.CrossEntropyLoss()
    
    # prepare model
    
    # https://github.com/team-approx-bayes/kpriors/blob/main/main.py#L159
    model = copy.deepcopy(base_model)
    optimiser = AdamReg(model, lr=learning_rate, weight_decay=prior_prec)
    optimiser.previous_weights = return_parameters(base_model)

    def select_memory_points(base_train_data, additional_memory_data):
        
        # Select points
        # https://github.com/team-approx-bayes/kpriors/blob/main/main.py#L144
        memory_points = {}
        # https://github.com/team-approx-bayes/kpriors/blob/main/utils.py#L63
        # https://github.com/team-approx-bayes/kpriors/blob/main/utils.py#L72
        memory_points['inputs'] = torch.cat((base_train_data[0], additional_memory_data[0]))
        memory_points['true_labels'] = torch.cat((base_train_data[1], additional_memory_data[1]))
        if use_cuda:
            memory_points['inputs'] = memory_points['inputs'].cuda()
        # https://github.com/team-approx-bayes/kpriors/blob/main/utils.py#L79
        memory_points['soft_labels'] = torch.softmax(base_model.forward(memory_points['inputs']), dim=-1)
        
        
        # Soft labels in K-priors, hard (true) labels in Replay
        if adaptation_method == "K-priors":
            memory_points['labels'] = memory_points['soft_labels']
        elif adaptation_method == "Replay":
            memory_points['labels'] = torch.nn.functional.one_hot(memory_points['true_labels'], num_classes=10)
        
        # Store past memory labels
        # https://github.com/team-approx-bayes/kpriors/blob/main/main.py#L230
        optimiser.memory_labels = memory_points['labels']

        return memory_points

    
    # https://github.com/team-approx-bayes/kpriors/blob/main/main.py#L218
    optimiser.prior_prec_old = prior_prec
    remove_data_bool = True

    # https://github.com/team-approx-bayes/kpriors/blob/main/main.py#L49
    train_batch_size = int(np.ceil(BATCH_SIZE * len(train_set) / (len(train_set) + len(forget_set))))
    forget_batch_size = int(np.ceil(BATCH_SIZE * len(forget_set) / (len(train_set) + len(forget_set))))

    num_steps = min(len(train_set) // train_batch_size, len(forget_set) // forget_batch_size)

    train_x, train_y = train_set.tensors[0], train_set.tensors[1]
    forget_x, forget_y = forget_set.tensors[0], forget_set.tensors[1]
    
    train_times = list()
    
    train_accs, test_accs, forget_accs = list(), list(), list()
    
    for epoch in range(EPOCHS):    
        
        # train
        
        train_time = 0
        
        start_time = time.time()

        model.train()
        for i in range(num_steps):
            
            # If remove_data task, then store the removed points too, for both K-priors and Replay
            
            # https://github.com/team-approx-bayes/kpriors/blob/main/main.py#L140
            # https://github.com/team-approx-bayes/kpriors/blob/main/main.py#L134
            base_train_data = (train_x[train_batch_size*i:train_batch_size*(i+1)], train_y[train_batch_size*i:train_batch_size*(i+1)])
            additional_memory_data = (forget_x[forget_batch_size*i:forget_batch_size*(i+1)], forget_y[forget_batch_size*i:forget_batch_size*(i+1)])
            
            # Load data for adaptation task
            # https://github.com/team-approx-bayes/kpriors/blob/main/main.py#L148
            # https://github.com/team-approx-bayes/kpriors/blob/main/data_generators.py#L67
            adapt_train_data = additional_memory_data
    
            memory_points = select_memory_points(base_train_data, additional_memory_data)
                
            # Train model
            # https://github.com/team-approx-bayes/kpriors/blob/main/main.py#L235
            
            train.train_model(
                model, optimiser, adapt_train_data, 
                num_epochs=1, # one epoch
                memory_data=memory_points,
                adaptation_method=adaptation_method, 
                remove_data_bool=remove_data_bool, 
                use_cuda=use_cuda
            )
        
        train_time += time.time() - start_time
            
        train_times.append(train_time)
        
        # test
            
        model.eval()
        with torch.no_grad():
            
            #
            
            x, y = train_set.tensors
            
            accs = list()
            
            for i in range(0, x.shape[0], BATCH_SIZE):
            
                output = model(x[i:i+BATCH_SIZE].cuda())

                predicted = torch.argmax(output.data, dim=-1)
                accs.append((predicted == y[i:i+BATCH_SIZE].cuda()).float().mean().detach().cpu().numpy())
            
            train_accs.append(np.mean(accs))
            
            #
            
            x, y = test_set.tensors
            
            accs = list()
            
            for i in range(0, x.shape[0], BATCH_SIZE):
            
                output = model(x[i:i+BATCH_SIZE].cuda())

                predicted = torch.argmax(output.data, dim=-1)
                accs.append((predicted == y[i:i+BATCH_SIZE].cuda()).float().mean().detach().cpu().numpy())
            
            test_accs.append(np.mean(accs))
            
            #

            x, y = forget_set.tensors

            for i in range(0, x.shape[0], BATCH_SIZE):

                output = model(x[i:i+BATCH_SIZE].cuda())

                predicted = torch.argmax(output.data, dim=-1)
                accs.append((predicted == y[i:i+BATCH_SIZE].cuda()).float().mean().detach().cpu().numpy())

            forget_accs.append(np.mean(accs))
        
        # save
        torch.save(model.state_dict(), os.path.join(save_dir, f'{(epoch+1):03d}.pt'))

    return train_times, train_accs, test_accs, forget_accs

In [9]:
for adaptation_method in adaptation_methods:

    results = list()
    
    for percentage in tqdm(PERCENTAGES):
        
        model = CNN().cuda()
        
        model.load_state_dict(torch.load('./weights/original/005.pt'))
        
        train_set, test_set, forget_set = load_deleted_dataset(DATA_DIR, percentage)
        
        train_times, train_accs, test_accs, forget_accs = fit(model, f'weights/kpriors/{adaptation_method}/{percentage}', train_set, test_set, forget_set)
        
        df = pd.DataFrame(zip(train_times, train_accs, test_accs, forget_accs), columns=['train_time', 'train_acc', 'test_acc', 'forget_acc'])
        df['epoch'] = range(1, EPOCHS+1)
        df['percentage'] = percentage
        
        results.append(df)

    results = pd.concat(results).set_index(['percentage', 'epoch'])
    
    results.to_csv(f'results/kpriors/{adaptation_method}.csv')

  0%|          | 0/11 [00:00<?, ?it/s]

  0%|          | 0/11 [00:00<?, ?it/s]

In [10]:
results

train_time  train_acc  test_acc  forget_acc
percentage epoch                                             
1          1        6.246719   0.974051  0.972843    0.973048
           2        6.432508   0.976794  0.979034    0.979198
           3        6.457139   0.974573  0.974840    0.974743
           4        6.407178   0.963786  0.961462    0.962067
           5        6.530234   0.972099  0.972943    0.972923
10         1       16.206944   0.971564  0.971246    0.972118
           2       16.340193   0.947275  0.953175    0.951722
           3       16.465451   0.958198  0.962660    0.962325
           4       16.547714   0.959938  0.961761    0.962450
           5       16.548919   0.972508  0.974042    0.974613
20         1       18.558873   0.967458  0.967851    0.964435
           2       19.057752   0.953354  0.953774    0.952444
           3       19.304313   0.947625  0.947085    0.945176
           4       19.363307   0.964354  0.965056    0.962436
           5       19.251297   0.963646  0.967153    0.964208
30         1       20.592468   0.633878  0.637081    0.623430
           2       20.892775   0.701495  0.711961    0.707156
           3       20.557113   0.911724  0.917831    0.913313
           4       20.637675   0.929265  0.930511    0.928403
           5       20.394286   0.956826  0.956669    0.956193
40         1       21.388638   0.914139  0.914736    0.915275
           2       21.023240   0.931611  0.931010    0.929004
           3       20.577450   0.708250  0.711861    0.711636
           4       20.893219   0.924056  0.923522    0.922713
           5       21.067666   0.749028  0.750499    0.749147
50         1       21.679158   0.917011  0.920427    0.915568
           2       21.708622   0.792344  0.790335    0.793865
           3       21.716053   0.508162  0.512780    0.508268
           4       21.696955   0.829091  0.839257    0.827638
           5       21.867433   0.446995  0.461861    0.454761
60         1       20.983034   0.161208  0.163938    0.159771
           2       21.960438   0.313958  0.320288    0.317650
           3       21.620485   0.263208  0.258886    0.259714
           4       20.987870   0.742000  0.746506    0.736852
           5       22.452072   0.469083  0.489617    0.478790
70         1       20.539524   0.397980  0.385184    0.382937
           2       21.192062   0.684447  0.682708    0.675027
           3       20.781398   0.372058  0.360224    0.356896
           4       21.042561   0.655029  0.646965    0.647582
           5       20.936912   0.370060  0.374700    0.371118
80         1       19.810648   0.424833  0.431609    0.427313
           2       19.841686   0.668167  0.667532    0.655923
           3       19.934380   0.309167  0.312400    0.310569
           4       20.188042   0.482750  0.476338    0.473697
           5       19.978985   0.205167  0.203874    0.201772
90         1       18.138252   0.468085  0.449281    0.447573
           2       18.247383   0.124834  0.119708    0.120003
           3       17.698911   0.345578  0.326777    0.332662
           4       18.578517   0.334940  0.336062    0.331209
           5       18.051911   0.103890  0.102436    0.102558
99         1        6.644415   0.098136  0.118710    0.117828
           2        6.956798   0.080592  0.084465    0.082661
           3        7.200098   0.077851  0.083766    0.082935
           4        7.026857   0.110197  0.104233    0.100922
           5        7.380495   0.098684  0.094349    0.096486